In [2]:
import numpy as np
import math
from termcolor import cprint


def get_daily_balance( total_demand, my_schedule, last_balance = 0):
    
    ndays = len( my_schedule )
    
    daily_demand = np.ones(ndays)*total_demand/ndays

    daily_balance = np.round(np.cumsum( np.append( last_balance,my_schedule) - np.append(0,daily_demand) ), 1)
    
    return( daily_balance[1:])

    
def my_print(my_schedule):
    for i in range(len(my_schedule)):
        if i<7:
            cprint(int(my_schedule[i]), 'blue', end=' ')
        else:
            cprint(int(my_schedule[i]), 'cyan', end=' ')
    print('')
    
    
def irrigation_schedule(new_weekly_demand
                  , old_weekly_demand      = 0
                  , old_plan               = [np.zeros(0), np.zeros(1)]
                  , n_days_forecast        = 11
                  , n_shift_init_days      = 0  # to be checked that is only >0 at first initialisation
                  , max_no_events_per_week = 5  # the most important
                  , min_runtime            = 3  # (hardware bound)
                  , max_runtime            = 16 # (hardware bound)
                  , max_irrig_time         = 14 # (physical bound) )
                 ):
    
    old_schedule = old_plan[0]
    old_balance  = old_plan[1]
    
    new_daily_demand  = new_weekly_demand / 7
    daily_demand_diff = (new_weekly_demand - old_weekly_demand) / 7
    
    # idea: we need to spread the new demand over the last four days. 
    # we have already booked six days. The difference due an updated demand will be taken over to the last four days.
    # additionally, we have the daily demand for the last four days
    # and we take the old balance from the sixth day over (balance=guthaben auf dem wasserkonto)
    
    # first calculate the number of days for which we do a recalculation
    if old_weekly_demand == 0 and len(old_schedule)==0: # at first initialisation
        n_new_days = n_days_forecast - n_shift_init_days
    elif (old_weekly_demand != 0 and (new_weekly_demand/old_weekly_demand >=3 or new_weekly_demand/old_weekly_demand <= 0.33)):
        n_new_days = n_days_forecast - 7 + 3
    elif (old_weekly_demand != 0 and (new_weekly_demand/old_weekly_demand >=2 or new_weekly_demand/old_weekly_demand <= 0.5)):
        n_new_days = n_days_forecast - 7 + 2
    else:
        n_new_days = n_days_forecast - 7 + 1
              
    n_old_days = n_days_forecast - n_new_days - n_shift_init_days
     
    # now calculate the demand that needs to be fulfilled in these n_new_days
    remaining_demand = round(n_old_days*daily_demand_diff + (n_new_days+n_shift_init_days)*new_daily_demand - old_balance[n_old_days],0)
    
    # now spread the demand: first calculate the number of new irrigation events
    my_max_time = max( min( max_runtime, max_irrig_time ), min_runtime ) # min_runtime is the strongest 
    
    n_irrig_events_new = min(max_no_events_per_week, math.ceil( remaining_demand / my_max_time ) )
    
    # now make the schedule
    new_schedule=np.append(old_schedule[1:n_old_days+1],np.zeros(n_new_days+n_shift_init_days))
    
    if n_irrig_events_new > 0:
        
        i_shift = 0

        irrig_days = np.where(np.floor(np.arange(n_new_days) % (n_new_days/n_irrig_events_new)) ==0)[0] + n_shift_init_days
  
        # we shift the scheduling if we don't have many events scheduled AND
        # if we just had one and the new demand is less than twice as high
        # or if the balance is still less than the daily demand and 
        # or we only have one event to schedule
    
        if ( old_weekly_demand > 0 and irrig_days[-1] != n_new_days-1 and
            ( 
               ( old_schedule[n_old_days] != 0  and  new_weekly_demand / old_weekly_demand <= 2) or 
               (-old_balance[n_old_days] < new_daily_demand and new_weekly_demand / old_weekly_demand <= 1) or
                n_irrig_events_new == 1
            )
           ): 
            i_shift = 1 
            
        for i in np.arange(0,n_irrig_events_new):
    
            new_schedule[irrig_days[i]+n_old_days+i_shift] = max( 
                math.ceil( ( remaining_demand - np.sum(new_schedule[n_old_days:irrig_days[i]+n_old_days] )) 
                                                    / ( n_irrig_events_new - i ) ), min_runtime)
            
    new_balance = get_daily_balance(new_weekly_demand*n_days_forecast/7, new_schedule,  old_balance[0])
        
    return np.array([new_schedule, new_balance])



#########################################################################
# put a series of weekly demands here:

weekly_demands=[21,22,25,30]

#########################################################################

plan_0=irrigation_schedule(weekly_demands[0], n_shift_init_days = 1)
print('initial demand:', weekly_demands[0])
my_print(plan_0[0,])
my_print(plan_0[1,])

my_plans=list()
my_plans.append(plan_0)

for i in np.arange(len(weekly_demands))[1:]:
 
    my_new_plan=irrigation_schedule(weekly_demands[i],weekly_demands[i-1], my_plans[i-1])
    my_plans.append(my_new_plan)
    
    print('new demand:', weekly_demands[i])
    my_print(my_plans[i][0])
    my_print(my_plans[i][1])

initial demand: 21
0 11 0 0 0 11 0 0 11 0 0 
-3 5 2 -1 -4 4 1 -2 6 3 0 
new demand: 22
11 0 0 0 11 0 8 0 0 8 0 
4 1 -1 -4 3 0 5 1 -1 3 0 
new demand: 25
0 0 0 11 0 8 0 8 0 0 7 
1 -2 -5 1 -2 2 -1 3 0 -3 0 
new demand: 30
0 0 11 0 8 0 14 0 0 13 0 
-3 -7 0 -4 -1 -5 4 0 -4 4 0 
